<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/Robotics_Asssesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 2 Python File**

In [ ]:
#!/usr/bin/env python3

# imports rospy module and twist message type
import rospy
from geometry_msgs.msg import Twist
import math

class MoveTurtleBot():

    # initialize class attributes
    def __init__(self):
        self.turtlebot_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1) # creates a publisher for cmd_vel topic, specify twist message, sets queue size
        self.cmd = Twist()   # initializes a twist object
        self.ctrl_c = False  # initializes a flag for shutdown
        self.rate = rospy.Rate(1) # initializes the rate of publising
        rospy.on_shutdown(self.shutdownhook) # sets the on_shutdown method to shutdownhook function when it is shutting down.

    def publish_once_in_cmd_vel(self):
        """
        This is because publishing in topics sometimes fails the first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important as it ensures the velocity command is published atleast once.
        """
        while not self.ctrl_c:
            connections = self.turtlebot_vel_publisher.get_num_connections()
            if connections > 0:
                self.turtlebot_vel_publisher.publish(self.cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()

    def shutdownhook(self):
        # works better than the rospy.is_shutdown()
        self.stop_turtlebot()
        self.ctrl_c = True

    def stop_turtlebot(self):
        rospy.loginfo("shutdown time! Stop the robot")
        self.cmd.linear.x = 0.0
        self.cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel()

    def MoveTurtleBot(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        self.cmd.linear.x = linear_speed
        self.cmd.angular.z = angular_speed

        i = 0
        rospy.loginfo("Moving TurtleBot!")
        while not self.ctrl_c and i <= moving_time:
            self.publish_once_in_cmd_vel()
            i = i+1
            self.rate.sleep()

        self.stop_turtlebot()

    def MoveRectangle(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        for _ in range(4):
            self.cmd.linear.x = linear_speed
            self.cmd.angular.z = 0.0
            i = 0

            rospy.loginfo("Moving TurtleBot in a straight line")
            while not self.ctrl_c and i <= moving_time:
                self.publish_once_in_cmd_vel()
                i = i+1
                self.rate.sleep()

            # Stop briefly
            self.stop_turtlebot()
            rospy.sleep(1.0)

            # turns 90 degrees to the right
            desired_angle_degrees = 90.0
            time_duration_seconds = 5.0

            # Convert degrees to radians
            desired_angle_radians = desired_angle_degrees*(math.pi/180.0) # (180 degrees = pi radians)

            # Calculate angular speed
            angular_speed = desired_angle_radians / time_duration_seconds

            # Set linear and angular speed values
            self.cmd.linear.x = 0.0
            self.cmd.angular.z = angular_speed

            # log and publish to topic
            rospy.loginfo("TurtleBot turns 90 degrees to the right!")
            self.publish_once_in_cmd_vel()
            rospy.sleep(5.0)


            # Stop briefly
            self.stop_turtlebot()
            rospy.sleep(1.0)

    def MoveCircle(self, duration=10, radius=1.0):

        # Calculate linear velocity to achieve a complete circle
        linear_speed = 2 * math.pi * radius / duration

        # Calculate angular velocity based on linear velocity and radius
        angular_speed = linear_speed / radius

        # Set values for linear and angular speed
        self.cmd.linear.x = linear_speed
        self.cmd.angular.z = angular_speed

        # log info
        rospy.loginfo(f"Moving TurtleBot in a circle for {duration} seconds.")
        start_time = rospy.get_time()

        while rospy.get_time() - start_time <= duration:
            self.turtlebot_vel_publisher.publish(self.cmd)
            self.rate.sleep()

        self.stop_turtlebot()
        rospy.sleep(1.0)


if __name__ == '__main__':
    rospy.init_node('move_turtlebot_test', anonymous=True)
    moveturtlebot_object = MoveTurtleBot()

    try:
        moveturtlebot_object.MoveRectangle(10)
        moveturtlebot_object.MoveCircle(10, 1.0)

    except rospy.ROSInterruptException:
        pass

**Question 2 Bash File**

In [ ]:
ARG1=$1

if [ "$ARG1" == "circle" ]; then
   echo "circling";
   rosrun rss_linux_pkg move_turtlebot_circle.py

elif [ "$ARG1" == 'forward_backward' ]; then
     echo "back and forth";
     rosrun rss_linux_pkg move_turtlebot_forward_backward.py

elif [ "$ARG1" == 'rectangle_circle' ]; then
     echo "rectangle and circle";
     rosrun rss_linux_pkg move_turtlebot_rectangle_circle.py
else
echo "Please enter one of the following;
circle
forward_backward
rectangle_circle"

fi

**Question 1**

In [ ]:
#!/usr/bin/env python3

# initiate variable
sum_result = 0

# define the range: 0 : 99
number_set = range(0,100)

# Calculate the sum from 0 to 99
for number_index in number_set:
    sum_result += number_index

# Print Result
print('Sum of numbers from 0 to 99:', sum_result)

**Question 1 Continued**
Detail what commands you used and for what purpose ?
1. touch
2. vim
3. chmod
4. ./

##Question 3

In [ ]:
#!/usr/bin/env python3

#import modules and packages
import rospy
from geometry_msgs.msg import Twist

#initialize the ROS node
rospy.init_node('osazeeogbeide_publisher_line')

#create a publisher for twist message
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

#rate at which message is published
rate = rospy.Rate(1)

#create a twist message with linear velocity along the x-axis
move = Twist()
move.linear.x = 0.5
move.angular.z = 0.0  # No angular velocity for moving straight

while not rospy.is_shutdown():
    pub.publish(move)
    rate.sleep()
####################################################################################################################

<launch>

<node name="oz_pub_line" pkg="rss_pubsub_pkg" type="osazeeogbeide_publisher_line.py" output="screen"/>

</launch>
###################################################################################################################
# line subscriber
#!/usr/bin/env python3

import rospy
from std_msgs.msg import Int32
from geometry_msgs.msg import Twist

def callback(msg):
    print("Linear velocity is")
    print(msg.linear)
    print("Angular velocity is")
    print(msg.angular)

#rospy initializes the subscriber node
rospy.init_node("osazeeogbeide_subscriber_line")

# declares that the node subscribes to the '/cmd_vel'topic  of type: Twist
sub = rospy.Subscriber("/cmd_vel", Twist, callback )

# keeps node from exiting until the node has been shut down
rospy.spin()
####################################################################################################################

# publisher
#!/usr/bin/env python3

#libraries to set robot velocity
import rospy
from geometry_msgs.msg import Twist

#Enables communication between the node: pengwang_publisher and ROS master
rospy.init_node("osazeeogbeide_publisher")

#Defines the publisher to publish on topic:/cmd_vel with message type: Twist
pub = rospy.Publisher("/cmd_vel", Twist, queue_size=1)

#Defines the frequency of running the loop
rate = rospy.Rate(1)

#Specifies the message to publish to '/cmd_vel' topic
move = Twist()
move.linear.x = 0.5
move.angular.z = 0.5

#Always check 'rospy.is_shutdown()'
while not rospy.is_shutdown():
    pub.publish(move)
    rate.sleep()
######################################################################################################################


##Question 4

In [ ]:
#!/usr/bin/env python3

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from std_msgs.msg import Int32
from tf.transformations import euler_from_quaternion
import math

class OsazeeOgbeidePubSub:
    def __init__(self):
        # Initialize ROS node
        rospy.init_node('osazeeogbeide_pubsub')

        # create Subscriber for Odometry
        self.odom_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

        # set frequency
        self.rate = rospy.Rate(1) # 1Hz

        # Create Twist message
        self.twist_msg = Twist()

        # Initialize variables
        self.start_position = None
        self.distance_threshold = 0.2 # Set the distance threshold to 0.2 meter
        self.is_stopped = False

    def odom_callback(self, odom_msg):

        # Extract position from Odometry message
        position = odom_msg.pose.pose.position
        x = position.x
        y = position.y

        # Calculate distance from start position
        if self.start_position is None:
            self.start_position = (x, y)
        distance = math.sqrt((x - self.start_position[0])**2 + (y - self.start_position[1])**2)

        # Check if distance threshold is reached
        if distance >= self.distance_threshold and not self.is_stopped:
            # Stop the robot
            self.twist_msg.linear.x = 0.0  # m/s
            self.twist_msg.angular.z = 0.0 # rads/s
            self.is_stopped = True

        elif distance < self.distance_threshold and self.is_stopped:
            # Reset velocity if distance is below threshold after stopping
            self.is_stopped = False

        # If not stopped, move the robot forward
        if not self.is_stopped:
            self.twist_msg.linear.x = 0.2  # 0.2 m/s
            self.twist_msg.angular.z = 0.0  # 0 rad/s

        # Publish Twist message
        self.cmd_vel_pub.publish(self.twist_msg)

        # Display distance. linear velocity, angular velocity covered
        rospy.loginfo("Distance covered: {:.2f} meters".format(distance))
        rospy.loginfo("Linear velocity: {:.2f} m/s".format(self.twist_msg.linear.x))
        rospy.loginfo("Angular velocity: {:.2f} rads/s".format(self.twist_msg.linear.z))

        # Sleep to maintain publishing rate
        self.rate.sleep()

    def run(self):
        # Keep the node running
        rospy.spin()

if __name__ == '__main__':
    osazeeogbeide_pubsub = OsazeeOgbeidePubSub()
    osazeeogbeide_pubsub.run()
#######################################################################################################################
<launch>

<node name="oz_pubsub_node" pkg="rss_pubsub_pkg" type="osazeeogbeide_pubsub.py" output="screen" />

</launch>


**Question 4 Method 2**

In [ ]:
#!/usr/bin/env python

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from math import sqrt

class OsazeePubSub:
    def __init__(self):
      # Initialize ROS node
        rospy.init_node('osazee_pubsub_node', anonymous=True)

        # create Subscriber for Odometry
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_publisher = rospy.Publisher("/cmd_vel", Twist, queue_size=10)

        # set frequency
        self.rate = rospy.Rate(1) # 1Hz

        # set maximum distance to run
        self.distance_to_run = 5.0  # Adjust this value to the distance you want the robot to run

        # set initial velocity
        self.linear_velocity = 0.2  # Adjust this value to set the linear velocity

        # set current distance
        self.current_distance = 0.0

        # set initial position and orientation
        self.initial_pose = None

    def odom_callback(self, odom):
        if self.initial_pose is None:

            self.initial_pose = odom.pose.pose.position

        # Extract position from Odometry message
        current_pose = odom.pose.pose.position

        # Calculate distance from initial position
        self.current_distance = sqrt((current_pose.x - self.initial_pose.x)**2 +
                                     (current_pose.y - self.initial_pose.y)**2)

        # Check if distance threshold is reached
        if self.current_distance >= self.distance_to_run:
            rospy.loginfo("Reached the desired distance. Stopping the robot.")
            self.stop_robot()
        else:
            self.move_robot()

    def move_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = self.linear_velocity
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def stop_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = 0.0
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

if __name__ == '__main__':
    try:
        osazee_pubsub = OsazeePubSub()
        rospy.spin()
    except rospy.ROSInterruptException:
        pass
###################################################################################################################

<launch>

    <node name="osazee_pubsub_node" pkg="rss_pubsub_pkg" type="osazee_pubsub.py" output="screen">
    </node>

</launch>
